# Paso 1: Preprocesamiento del Dataset

El primer paso en nuestro proceso es el preprocesamiento del dataset. Este dataset se encuentra en formato CSV y requiere de varias transformaciones antes de poder ser utilizado para entrenar nuestro modelo.

1. Eliminación de columnas no útiles, valores nulos y duplicados: Algunas columnas como 'Unnamed: 0' y 'Flow ID' no aportan información relevante para el entrenamiento del modelo, por lo que serán eliminadas. Los valores nulos (o NaN) y los elementos duplicados pueden afectar la precisión de nuestro modelo, por lo que también serán eliminados.

In [1]:
import pandas as pd

#Cargar el dataset, eliminar columnas irrelevantes, valores nulos y elementos repetidos
df = pd.read_csv('DataSets//CICDDoS2019//01-12//Syn(2).csv')
df = df.drop(['Unnamed: 0', 'Flow ID'], axis=1)
df = df.dropna()
df = df.drop_duplicates()

2. Codificación de datos categóricos: Tenemos varias columnas (como la IP) que contienen datos categóricos que deben ser codificados, por ejemplo, numéricamente.

In [2]:
import ipaddress
from sklearn.preprocessing import LabelEncoder

#Codificar columnas categóricas
le = LabelEncoder()

#IP
df[' Source IP'] = df[' Source IP'].apply(lambda ip: int(ipaddress.IPv4Address(ip)))
df[' Destination IP'] = df[' Destination IP'].apply(lambda ip: int(ipaddress.IPv4Address(ip)))

#Fecha
df[' Timestamp'] = pd.to_datetime(df[' Timestamp'])
df.insert(5, ' Year', df[' Timestamp'].dt.year)
df.insert(6, ' Month', df[' Timestamp'].dt.month)
df.insert(7, ' Day', df[' Timestamp'].dt.day)
df.insert(8, ' Hour', df[' Timestamp'].dt.hour)
df.insert(9, ' Minute', df[' Timestamp'].dt.minute)
df.insert(10, ' Second', df[' Timestamp'].dt.second)
df = df.drop(' Timestamp', axis=1)

#SimilarHTTP
df['SimillarHTTP'] = le.fit_transform(df['SimillarHTTP'])

3. Transformación de tipos de datos: Algunas columnas pueden tener tipos de datos que ocupan más memoria de la necesaria. Por eso, podemos convertir columnas de tipo float64 a float32 para ahorrar memoria. 

In [3]:
# Convertir los tipos de datos de las columnas
for col in df.columns:
    if df[col].dtype == 'float64':
        df[col] = df[col].astype('float32')

4. Guardar el dataset preprocesado: Finalmente, guardaremos el dataset preprocesado en disco en formato Parquet, que es un formato de archivo columnar que proporciona almacenamiento eficiente.

In [4]:
# Guardar el dataset en formato Parquet
df.to_parquet('SynComprimido.parquet')

5. Preprocesamiento de todos los conjuntos de datos disponibles: Aquí, hemos repetido los pasos de las fases anteriores, pero esta vez con todos los conjuntos de datos disponibles a nuestra disposición.

In [1]:
import pandas as pd
import os
import ipaddress
from sklearn.preprocessing import LabelEncoder

# Lista de todos los directorios que contienen archivos CSV
data_dirs = ['DataSets//CICDDoS2019//01-12//', 'DataSets//CICDDoS2019//03-11//']

# Inicializar el codificador de etiquetas
le = LabelEncoder()

for data_dir in data_dirs:
    files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

    for file in files:
        # Cargar el dataset, eliminar columnas irrelevantes, valores nulos y elementos repetidos
        df = pd.read_csv(data_dir + file, low_memory=False)
        df = df.drop(['Unnamed: 0', 'Flow ID'], axis=1)
        df = df.dropna()
        df = df.drop_duplicates()

        # Transformar las columnas de IP
        df[' Source IP'] = df[' Source IP'].apply(lambda ip: int(ipaddress.IPv4Address(ip)))
        df[' Destination IP'] = df[' Destination IP'].apply(lambda ip: int(ipaddress.IPv4Address(ip)))

        # Transformar la columna de fecha
        df[' Timestamp'] = pd.to_datetime(df[' Timestamp'])
        df.insert(5, ' Year', df[' Timestamp'].dt.year)
        df.insert(6, ' Month', df[' Timestamp'].dt.month)
        df.insert(7, ' Day', df[' Timestamp'].dt.day)
        df.insert(8, ' Hour', df[' Timestamp'].dt.hour)
        df.insert(9, ' Minute', df[' Timestamp'].dt.minute)
        df.insert(10, ' Second', df[' Timestamp'].dt.second)
        df = df.drop(' Timestamp', axis=1)

        #Transformar la columna de SimillarHTTP
        df['SimillarHTTP'] = le.fit_transform(df['SimillarHTTP'])

        # Convertir los tipos de datos de las columnas float64 a float32
        for col in df.columns:
            if df[col].dtype == 'float64':
                df[col] = df[col].astype('float32')

        # Guardar el DataFrame en un archivo Parquet con el mismo nombre que el archivo CSV original
        df.to_parquet('DataSets//CICDDoS2019//Parquet//' + file.replace('.csv', '.parquet'))

2. Contar las ocurrencias de 'Benign' y tipos de ataques en múltiples archivos .parquet: 
    Este script de Python cuenta el número de filas que tienen 'Benign' y diferentes tipos de ataques en la columna 'Label' en cada archivo .parquet en una carpeta especificada.

In [2]:
import os
import pandas as pd

# Especifica la ruta a la carpeta que contiene los archivos .parquet
ruta_carpeta = 'DataSets//CICDDoS2019//Parquet'
archivos_parquet = [f for f in os.listdir(ruta_carpeta) if f.endswith('.parquet')]

# Para cada archivo .parquet en la carpeta
for archivo in archivos_parquet:
    df = pd.read_parquet(os.path.join(ruta_carpeta, archivo))
    conteo = df[' Label'].value_counts()
    
    # Imprime el nombre del archivo y el conteo
    print(f'Archivo: {archivo}')
    print(conteo)
    print('\n')

Archivo: DrDoS_DNS.parquet
 Label
DrDoS_DNS    5071002
BENIGN          3380
Name: count, dtype: int64


Archivo: DrDoS_LDAP.parquet
 Label
DrDoS_LDAP    2179928
BENIGN           1602
Name: count, dtype: int64


Archivo: DrDoS_MSSQL.parquet
 Label
DrDoS_MSSQL    4522489
BENIGN            1995
Name: count, dtype: int64


Archivo: DrDoS_NetBIOS.parquet
 Label
DrDoS_NetBIOS    4093273
BENIGN              1705
Name: count, dtype: int64


Archivo: DrDoS_NTP.parquet
 Label
DrDoS_NTP    1202639
BENIGN         14337
Name: count, dtype: int64


Archivo: DrDoS_SNMP.parquet
 Label
DrDoS_SNMP    5159863
BENIGN           1502
Name: count, dtype: int64


Archivo: DrDoS_SSDP.parquet
 Label
DrDoS_SSDP    2610610
BENIGN            762
Name: count, dtype: int64


Archivo: DrDoS_UDP.parquet
 Label
DrDoS_UDP    3134643
BENIGN          2151
Name: count, dtype: int64


Archivo: LDAP.parquet
 Label
LDAP       1905191
NetBIOS     202919
BENIGN        5111
Name: count, dtype: int64


Archivo: MSSQL.parquet
 Lab

3. Normalización y entreno SVM

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import svm
from sklearn.metrics import classification_report

# Carga el conjunto de datos desde un archivo Parquet
df = pd.read_parquet('DataSets//CICDDoS2019//Parquet//Syn(1).parquet')

# Reemplaza todas las etiquetas de ataque por 'Ataque'
df[' Label'] = df[' Label'].apply(lambda x: 'Attack' if x != 'BENIGN' else x)

df.to_parquet('DataSets//CICDDoS2019//Parquet//Syn(3).parquet')

# Crea un codificador de etiquetas y ajústalo a las etiquetas de tus datos
encoder = LabelEncoder()
encoder.fit(df[' Label'])

# Transforma las etiquetas a números
df[' Label'] = encoder.transform(df[' Label'])

# Divide los datos en características (X) y etiquetas (y)
X = df.drop(' Label', axis=1)
y = df[' Label']

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crea un escalador y ajustalo a los datos de entrenamiento
scaler = StandardScaler()
scaler.fit(X_train)

# Normaliza los conjuntos de entrenamiento y prueba
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Crea y entrena la SVM
clf = svm.SVC()
clf.fit(X_train, y_train)

# Haz predicciones con el conjunto de prueba
y_pred = clf.predict(X_test)

# Imprime un informe de clasificación
print(classification_report(y_test, y_pred))

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [4]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
from pyarrow import parquet as pq

# Carga los datos del archivo .parquet
data = pq.read_table('DataSets//CICDDoS2019//Parquet//Syn(1).parquet').to_pandas()

# Normaliza los datos
scaler = preprocessing.StandardScaler()
data_normalized = pd.DataFrame(scaler.fit_transform(data.drop(' Label', axis=1)), columns=data.columns.drop(' Label'))

# Transforma las etiquetas a números
le = preprocessing.LabelEncoder()
data_normalized[' Label'] = le.fit_transform(data[' Label'])

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data_normalized.drop(' Label', axis=1), data_normalized[' Label'], test_size=0.2, random_state=42)

# Entrena una SVM
clf = svm.SVC()
clf.fit(X_train, y_train)

# Predice los ataques en el conjunto de prueba
y_pred = clf.predict(X_test)

# Imprime un informe de clasificación
print(classification_report(y_test, y_pred))

ValueError: Input X contains infinity or a value too large for dtype('float64').